In [1]:
import pandas as pd, numpy as np
import json

In [2]:
progress = pd.read_csv('historical_progresses_apr.csv')

In [3]:
progress[progress['subjectCode'] == 'en'].groupby(['subjectCode', 'studentName', 'lessonId']).agg({'score': [np.max], 'lessonOrder': lambda x: np.mean(x)})

score lessonOrder
                                   amax    <lambda>
subjectCode studentName lessonId                   
en          Aadharsha   balloon35    30       189.0
                        balloon6     50        62.0
                        en0400      100        39.0
                        en0500       44        59.0
                        en0501       54        60.0
...                                 ...         ...
            Yashwanth S en0902       84        85.0
            kavitha     ambulance    63         4.0
                        en0000       72         1.0
                        en0001       45         2.0
                        en0002      100         3.0

[2289 rows x 2 columns]

In [4]:
students = {}
for row in progress.itertuples():
    score = row[7]
    student = row[10]
    subject = row[12]
    lesson_order = row[14]
    date_time = row[13]
    if(student == 'nan' or student == '2'):
        print(row)
    if student not in students:
        students[student] = {subject: {lesson_order: (score, date_time)}}
    elif subject not in students[student]:
        students[student][subject] = {lesson_order: (score, date_time)}
    elif lesson_order not in students[student][subject]:
        students[student][subject][lesson_order] = (score, date_time)
    elif students[student][subject][lesson_order][0] < score:
        students[student][subject][lesson_order] = (score, date_time)


Pandas(Index=3597, id='ziTLajct6Hhzj9WcFybA', progressId='ff1ef8a1-5b25-4727-b11a-304ae688fcb8', chapterId='en08', chapterName='Phonic Sense', lessonId='balloon9', lessonName='Challenge', score=90, schoolId='QwdOto9bdsSGLwf6G7Kd', sectionId='hh7Wczfy3yXOeK9kBiQI', studentId='2', studentName='Kushi', subjectCode='en', date='Thu Mar 31 2022 13:38:05 GMT+0530 (India Standard Time)', lessonOrder=77)
Pandas(Index=3598, id='T8IrSN4A86MKiKqJSILJ', progressId='ff1ef8a1-5b25-4727-b11a-304ae688fcb8', chapterId='en08', chapterName='Phonic Sense', lessonId='en0804', lessonName='Beginning Sounds - 5', score=94, schoolId='QwdOto9bdsSGLwf6G7Kd', sectionId='hh7Wczfy3yXOeK9kBiQI', studentId='2', studentName='Kushi', subjectCode='en', date='Thu Mar 31 2022 13:38:08 GMT+0530 (India Standard Time)', lessonOrder=78)
Pandas(Index=3599, id='kUd2QG3QgvNgdJmCLDJa', progressId='ff1ef8a1-5b25-4727-b11a-304ae688fcb8', chapterId='en08', chapterName='Phonic Sense', lessonId='en0805', lessonName='Medial Sounds', sco

In [5]:
print(students.keys())

dict_keys(['CV01', 'CV08', 'CV09', 'CV02', 'CV04', 'CV03', nan, 'CV10', 'CV07', 'CV06', 'SH29', 'SH25', 'SH21', 'SH33', 'SH20', 'SH22', 'SH28', 'SH19', 'SH18', 'SH27', 'SH24', 'SH17', 'SH13', 'SH15', 'KH24', 'KH20', 'KH31', 'KH25', 'KH18', 'KH22', 'KH23', 'KH26', 'KH27', 'KH33', 'KH34', 'KH29', 'KH28', 'KH21', 'KH19', 'KH13', 'KH08', 'KH05', 'KH12', 'KH07', '1', 'MA24', 'MA27', 'MA26', 'MA03', 'MA08', 'aq2', 'MA15', 'MA20', 'MA21', 'MA17', 'MA23', 'MA22', 'awsd', 'MA10', 'MA14', 'MA11', 'MA18', 'MA12', 'UD14', 'UD08', 'UD10', 'UD07', 'UD09', 'UD12', 'UD11', '2', 'UD13', 'UD05', 'UD03', 'UD04', 'UD06', 'UD02', 'UD01', 'BA26', 'AM14', 'AM13', 'AM18', 'AM19', 'AM20', 'AM17', 'AM15', 'AM16', 'AM22', 'AM11', 'AM10', 'AM25', 'AM04', 'AM23', 'AM21', 'AM03', 'AM01', 'AM12', 'AM02', 'AM05', 'AM08', 'AM24', 'AM09', 'AM06', 'CH02', 'CH06', 'CH09', 'CH08', 'CH01', 'CH07', 'CH04', 'CH10', 'CH05', 'CH03', 'CH24', 'CH22', 'CH15', 'CH21', 'CH11', 'CH17', 'CH20', 'CH16', 'CH12', 'CH13', 'CH14', 'DM03',

In [14]:
max_lessons = {
    'kn': 379/2,
    'en': 303/2,
    'maths': 404/2
}
# 1, 1.5, 2.5, 3, 4, 5, 6,7,7
month = 7.1
total_months = 10
alpha_avg = 0
for student, course in students.items():
    course_avg = {
        'kn': 0,
        'en': 0,
        'maths': 0
    }
    lesson_details = []
    for course_name, lessons in course.items():
        if course_name in max_lessons:
            lesson_avg = 0
            score_avg = 0
            for lesson_order, score in lessons.items():
                lesson_avg += lesson_order
                score_avg += score[0]
            lesson_avg = lesson_avg / max(max_lessons[course_name]/total_months, len(lessons))
            ideal_lesson_avg = max_lessons[course_name] * (month - 0.5)/total_months
            score_avg /= len(lessons)
            # print(course_name, lesson_avg, ideal_lesson_avg, lesson_avg/ideal_lesson_avg, score_avg, len(lessons), max_lessons[course_name]/total_months)
            course_avg[course_name] = lesson_avg/ideal_lesson_avg * (score_avg + 15) / 100
            lesson_details.append({'course': course_name, 'lesson_num': lesson_order, 'score': score[0], 'date': score[1]})
    alpha = (course_avg['kn'] + course_avg['en'] + course_avg['maths']) / 3
    print(student, '\t', alpha, '\t', json.dumps(lesson_details))
    alpha_avg += alpha
alpha_avg = alpha_avg / len(students)

CV01 	 0.6074626889408065 	 [{"course": "kn", "lesson_num": 180, "score": 74, "date": "Thu Mar 31 2022 13:12:50 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 99, "score": 96, "date": "Thu Apr 07 2022 12:49:20 GMT+0530 (India Standard Time)"}, {"course": "en", "lesson_num": 236, "score": 25, "date": "Thu Apr 07 2022 11:24:56 GMT+0530 (India Standard Time)"}]
CV08 	 0.059345868580257366 	 [{"course": "en", "lesson_num": 290, "score": 78, "date": "Thu Apr 07 2022 09:58:21 GMT+0530 (India Standard Time)"}]
CV09 	 0.023744948752300977 	 [{"course": "maths", "lesson_num": 176, "score": 94, "date": "Thu Apr 07 2022 09:58:19 GMT+0530 (India Standard Time)"}]
CV02 	 0.0 	 []
CV04 	 0.04594016882436319 	 [{"course": "en", "lesson_num": 236, "score": 20, "date": "Thu Apr 07 2022 11:51:05 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 102, "score": 100, "date": "Thu Apr 07 2022 12:39:56 GMT+0530 (India Standard Time)"}]
CV03 	 0.4719673754089027 	 [{"cour

In [15]:
print(alpha_avg)

0.3696321658669141
